In [281]:
from textblob.classifiers import NLTKClassifier,NaiveBayesClassifier,DecisionTreeClassifier,PositiveNaiveBayesClassifier,MaxEntClassifier
import pandas as pd
import re

path = '/Users/Jared/Documents/CSCE489/489Project/Turk/'

In [282]:
turk_df = pd.read_csv(path + 'Batch_2606965_batch_results.csv') 
turk_df.head()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,RequesterFeedback,WorkTimeInSeconds,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.status_id,Input.content,Answer.sentiment,Approve,Reject
0,301KG0KX9CFC0MHLW3ECT80VTS3H2D,3WROYBU7BMMRCGD41SQ9YGNC31GDCO,Preseidential Election Sentiment Analysis,Analyze the sentiment of the provided Facebook...,sentiment election Donald Trump Hilary Clinton...,$0.02,Wed Nov 23 21:44:16 PST 2016,3,BatchId:2606965;,172800,...,NaN,3,100% (143/143),100% (143/143),100% (143/143),13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Positive,NaN,NaN
1,301KG0KX9CFC0MHLW3ECT80VTS3H2D,3WROYBU7BMMRCGD41SQ9YGNC31GDCO,Preseidential Election Sentiment Analysis,Analyze the sentiment of the provided Facebook...,sentiment election Donald Trump Hilary Clinton...,$0.02,Wed Nov 23 21:44:16 PST 2016,3,BatchId:2606965;,172800,...,NaN,3,100% (349/349),100% (349/349),100% (349/349),13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Positive,NaN,NaN
2,301KG0KX9CFC0MHLW3ECT80VTS3H2D,3WROYBU7BMMRCGD41SQ9YGNC31GDCO,Preseidential Election Sentiment Analysis,Analyze the sentiment of the provided Facebook...,sentiment election Donald Trump Hilary Clinton...,$0.02,Wed Nov 23 21:44:16 PST 2016,3,BatchId:2606965;,172800,...,NaN,7,100% (132/132),100% (132/132),100% (132/132),13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Neutral,NaN,NaN
3,301KG0KX9CFC0MHLW3ECT80VTS42HZ,3WROYBU7BMMRCGD41SQ9YGNC31GDCO,Preseidential Election Sentiment Analysis,Analyze the sentiment of the provided Facebook...,sentiment election Donald Trump Hilary Clinton...,$0.02,Wed Nov 23 21:44:36 PST 2016,3,BatchId:2606965;,172800,...,NaN,51,100% (147/147),100% (147/147),100% (147/147),131459315949_10154056577745950,From Barack Obama's 2003 Senate campaign to Hi...,Positive,NaN,NaN
4,301KG0KX9CFC0MHLW3ECT80VTS42HZ,3WROYBU7BMMRCGD41SQ9YGNC31GDCO,Preseidential Election Sentiment Analysis,Analyze the sentiment of the provided Facebook...,sentiment election Donald Trump Hilary Clinton...,$0.02,Wed Nov 23 21:44:36 PST 2016,3,BatchId:2606965;,172800,...,NaN,3,100% (99/99),100% (99/99),100% (99/99),131459315949_10154056577745950,From Barack Obama's 2003 Senate campaign to Hi...,Neutral,NaN,NaN


In [283]:
relevant_df = turk_df[['Input.status_id','Input.content','Answer.sentiment']]
relevant_df.columns = ['status_id','content','sentiment']
relevant_df.head()

,status_id,content,sentiment
0,13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Positive
1,13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Positive
2,13652355666_10154036300320667,Thousands of demonstrators filled the streets ...,Neutral
3,131459315949_10154056577745950,From Barack Obama's 2003 Senate campaign to Hi...,Positive
4,131459315949_10154056577745950,From Barack Obama's 2003 Senate campaign to Hi...,Neutral


In [284]:
df1 = relevant_df[['status_id','content']]
df2 = relevant_df[['status_id','sentiment']]

In [285]:
def remove_non_ascii(text):
    return re.sub(r'[^\x00-\x7F]',' ', text)

In [286]:
df1 = df1.groupby(df1.status_id).first()
#df1.index.name = None
df1.head()

,content
status_id,
131459315949_10152751596575950,"One opinion: ""Hillary Clinton won because all ..."
131459315949_10152753624975950,Second fiddle to Hillary Clinton in the debate...
131459315949_10152754227800950,Donald J. Trump may not attend the next Republ...
131459315949_10152758428025950,Conservatives have criticized Donald J. Trump ...
131459315949_10152766742570950,Hillary Clinton has promised to take on the Na...


In [287]:
df1['content'] = df1.content.apply(remove_non_ascii)
df1.head()

,content
status_id,
131459315949_10152751596575950,"One opinion: ""Hillary Clinton won because all ..."
131459315949_10152753624975950,Second fiddle to Hillary Clinton in the debate...
131459315949_10152754227800950,Donald J. Trump may not attend the next Republ...
131459315949_10152758428025950,Conservatives have criticized Donald J. Trump ...
131459315949_10152766742570950,Hillary Clinton has promised to take on the Na...


In [288]:
def choose_sentiment(label):
    if label.count('Positive') >= 2:
        return 'Positive'
    elif label.count('Neutral') >= 2:
        return 'Neutral'
    elif label.count('Negative') >= 2:
        return 'Negative'
    else:
        return 'Inconclusive'

In [289]:
df2 = df2.groupby(df2.status_id).sum()
df2.head()

,sentiment
status_id,
131459315949_10152751596575950,NeutralNegativeNegative
131459315949_10152753624975950,NeutralPositiveNegative
131459315949_10152754227800950,PositiveNeutralNeutral
131459315949_10152758428025950,NegativeNegativeNeutral
131459315949_10152766742570950,NeutralNegativeNeutral


In [290]:
df2['sentiment'] = df2.sentiment.apply(choose_sentiment)
df2.head()

,sentiment
status_id,
131459315949_10152751596575950,Negative
131459315949_10152753624975950,Inconclusive
131459315949_10152754227800950,Neutral
131459315949_10152758428025950,Negative
131459315949_10152766742570950,Neutral


In [291]:
df2 = df2[df2.sentiment != 'Inconclusive']
df2.head()

,sentiment
status_id,
131459315949_10152751596575950,Negative
131459315949_10152754227800950,Neutral
131459315949_10152758428025950,Negative
131459315949_10152766742570950,Neutral
131459315949_10152791196665950,Positive


In [292]:
merged_df = df1.merge(df2,how='right',left_index=True,right_index=True)
merged_df.head()

,content,sentiment
status_id,,
131459315949_10152751596575950,"One opinion: ""Hillary Clinton won because all ...",Negative
131459315949_10152754227800950,Donald J. Trump may not attend the next Republ...,Neutral
131459315949_10152758428025950,Conservatives have criticized Donald J. Trump ...,Negative
131459315949_10152766742570950,Hillary Clinton has promised to take on the Na...,Neutral
131459315949_10152791196665950,A look at 5 crucial issues in the Canadian ele...,Positive


In [293]:
def classify_post (row):
   status = row['content'].lower()
   if (('donald' in status) or ('trump' in status)) and (('hillary' in status) or ('clinton' in status)) :
      return 'other'
   elif ('donald' in status) or ('trump' in status) :
      return 'trump'
   elif ('hillary' in status) or ('clinton' in status) :
      return 'clinton'
   else:
      return 'other'

In [294]:
labeled_df = merged_df
labeled_df['candidate_label'] = merged_df.apply (lambda row: classify_post (row),axis=1)
labeled_df.head()

,content,sentiment,candidate_label
status_id,,,
131459315949_10152751596575950,"One opinion: ""Hillary Clinton won because all ...",Negative,clinton
131459315949_10152754227800950,Donald J. Trump may not attend the next Republ...,Neutral,trump
131459315949_10152758428025950,Conservatives have criticized Donald J. Trump ...,Negative,trump
131459315949_10152766742570950,Hillary Clinton has promised to take on the Na...,Neutral,clinton
131459315949_10152791196665950,A look at 5 crucial issues in the Canadian ele...,Positive,other


In [295]:
trump_df = labeled_df[labeled_df.candidate_label == 'trump']
trump_df.head()

,content,sentiment,candidate_label
status_id,,,
131459315949_10152754227800950,Donald J. Trump may not attend the next Republ...,Neutral,trump
131459315949_10152758428025950,Conservatives have criticized Donald J. Trump ...,Negative,trump
131459315949_10152871191975950,Donald J. Trump said that he wasn't sure about...,Neutral,trump
131459315949_10152978451960950,Dr. Ben Carson has taken the lead in the Repub...,Positive,trump
131459315949_10153076683820950,We re fighting a very politically correct...,Neutral,trump


In [296]:
clinton_df = labeled_df[labeled_df.candidate_label == 'clinton']
clinton_df.head()

,content,sentiment,candidate_label
status_id,,,
131459315949_10152751596575950,"One opinion: ""Hillary Clinton won because all ...",Negative,clinton
131459315949_10152766742570950,Hillary Clinton has promised to take on the Na...,Neutral,clinton
131459315949_10152957933900950,Katy Perry performed at a rally for Hillary Cl...,Positive,clinton
131459315949_10153018635795950,"Politics Alert: In a new poll, 62 percent said...",Positive,clinton
131459315949_10153173878100950,With voters mobbing Bernie Sanders at events i...,Neutral,clinton


In [297]:
other_df = labeled_df[labeled_df.candidate_label == 'other']
other_df.head()

,content,sentiment,candidate_label
status_id,,,
131459315949_10152791196665950,A look at 5 crucial issues in the Canadian ele...,Positive,other
131459315949_10152983500645950,After an election in June saw nearly 60 percen...,Positive,other
131459315949_10153139941305950,Donald J. Trump referenced the male anatomy to...,Negative,other
131459315949_10153248450690950,Get a taste of a Hillary Clinton rally in full...,Neutral,other
131459315949_10153257668255950,"""So what s really at stake in this year s ...",Neutral,other


In [298]:
tuple_arr = list(zip(labeled_df.content,labeled_df.sentiment))
size = len(tuple_arr)
halfway_pt = size/2

print size

train = tuple_arr[:halfway_pt]
test = tuple_arr[halfway_pt:]
print len(train)
print len(test)

847
423
424


In [299]:
trump_tuple_arr = list(zip(trump_df.content,trump_df.sentiment))
size = len(trump_tuple_arr)
halfway_pt = size/2

print size

trump_train = trump_tuple_arr[:halfway_pt]
trump_test = trump_tuple_arr[halfway_pt:]
print len(trump_train)
print len(trump_test)

491
245
246


In [300]:
clinton_tuple_arr = list(zip(clinton_df.content,clinton_df.sentiment))
size = len(clinton_tuple_arr)
halfway_pt = size/2

print size

clinton_train = clinton_tuple_arr[:halfway_pt]
clinton_test = clinton_tuple_arr[halfway_pt:]
print len(clinton_train)
print len(clinton_test)

188
94
94


In [301]:
other_tuple_arr = list(zip(other_df.content,other_df.sentiment))
size = len(other_tuple_arr)
halfway_pt = size/2

print size

other_train = other_tuple_arr[:halfway_pt]
other_test = other_tuple_arr[halfway_pt:]
print len(other_train)
print len(other_test)

168
84
84


In [302]:
nltk_classifier = NLTKClassifier(trump_train)
print 'Done Training Trump NLTK Classifier!'

Done Training Trump NLTK Classifier!


In [303]:
nltk_classifier = NLTKClassifier(clinton_train)
print 'Done Training Clinton NLTK Classifier!'

Done Training Clinton NLTK Classifier!


In [304]:
nltk_classifier = NLTKClassifier(other_train)
print 'Done Training Other NLTK Classifier!'

Done Training Other NLTK Classifier!


In [305]:
naivebayes_classifier = NaiveBayesClassifier(trump_train)
print 'Done Training Trump Naive Bayes Classifier!'

Done Training Trump Naive Bayes Classifier!


In [306]:
naivebayes_classifier = NaiveBayesClassifier(clinton_train)
print 'Done Training Clinton Naive Bayes Classifier!'

Done Training Clinton Naive Bayes Classifier!


In [307]:
naivebayes_classifier = NaiveBayesClassifier(other_train)
print 'Done Training Other Naive Bayes Classifier!'

Done Training Other Naive Bayes Classifier!


In [308]:
decisiontree_classifier = DecisionTreeClassifier(trump_train)
print 'Done Training Trump Decision Tree Classifier!'

Done Training Trump Decision Tree Classifier!


In [309]:
decisiontree_classifier = DecisionTreeClassifier(clinton_train)
print 'Done Training Clinton Decision Tree Classifier!'

Done Training Clinton Decision Tree Classifier!


In [310]:
decisiontree_classifier = DecisionTreeClassifier(other_train)
print 'Done Training Other Decision Tree Classifier!'

Done Training Other Decision Tree Classifier!


In [311]:
#positivenaivebayes_classifier = PositiveNaiveBayesClassifier(trump_train)
#print 'Done Training Positive Naive Bayes Classifier!'

In [312]:
maxent_classifier = MaxEntClassifier(trump_train)
print 'Done Training Trump Max Ent Classifier!'

Done Training Trump Max Ent Classifier!


In [313]:
maxent_classifier = MaxEntClassifier(clinton_train)
print 'Done Training Clinton Max Ent Classifier!'

Done Training Clinton Max Ent Classifier!


In [314]:
maxent_classifier = MaxEntClassifier(other_train)
print 'Done Training Other Max Ent Classifier!'

Done Training Other Max Ent Classifier!


In [315]:
print "Trump Accuracy"
#nltk_classifier_accuracy = nltk_classifier.accuracy(test)
naivebayes_classifier_accuracy = naivebayes_classifier.accuracy(trump_test)
decisiontree_classifier_accuracy = decisiontree_classifier.accuracy(trump_test)
#positivenaivebayes_classifier_accuracy = positivenaivebayes_classifier.accuracy(test)
maxent_classifier_accuracy = maxent_classifier.accuracy(trump_test)

#print "NLTK Accuracy: ", nltk_classifier_accuracy
print "Naive Bayes Accuracy: ", naivebayes_classifier_accuracy
print "Decision Tree Accuracy: ", decisiontree_classifier_accuracy
#print "Positive Naive Bayes Accuracy: ", positivenaivebayes_classifier_accuracy
print "Maxent Accuracy: ", maxent_classifier_accuracy

print "Clinton Accuracy"
#nltk_classifier_accuracy = nltk_classifier.accuracy(test)
naivebayes_classifier_accuracy = naivebayes_classifier.accuracy(clinton_test)
decisiontree_classifier_accuracy = decisiontree_classifier.accuracy(clinton_test)
#positivenaivebayes_classifier_accuracy = positivenaivebayes_classifier.accuracy(test)
maxent_classifier_accuracy = maxent_classifier.accuracy(clinton_test)

#print "NLTK Accuracy: ", nltk_classifier_accuracy
print "Naive Bayes Accuracy: ", naivebayes_classifier_accuracy
print "Decision Tree Accuracy: ", decisiontree_classifier_accuracy
#print "Positive Naive Bayes Accuracy: ", positivenaivebayes_classifier_accuracy
print "Maxent Accuracy: ", maxent_classifier_accuracy

print "Other Accuracy"
#nltk_classifier_accuracy = nltk_classifier.accuracy(test)
naivebayes_classifier_accuracy = naivebayes_classifier.accuracy(other_test)
decisiontree_classifier_accuracy = decisiontree_classifier.accuracy(other_test)
#positivenaivebayes_classifier_accuracy = positivenaivebayes_classifier.accuracy(test)
maxent_classifier_accuracy = maxent_classifier.accuracy(other_test)

#print "NLTK Accuracy: ", nltk_classifier_accuracy
print "Naive Bayes Accuracy: ", naivebayes_classifier_accuracy
print "Decision Tree Accuracy: ", decisiontree_classifier_accuracy
#print "Positive Naive Bayes Accuracy: ", positivenaivebayes_classifier_accuracy
print "Maxent Accuracy: ", maxent_classifier_accuracy

Trump Accuracy
  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.286
             2          -0.09205        1.000
             3          -0.02423        1.000
             4          -0.01154        1.000
             5          -0.00671        1.000
             6          -0.00421        1.000
             7          -0.00274        1.000
             8          -0.00180        1.000
             9          -0.00119        1.000
            10          -0.00079        1.000
            11          -0.00052        1.000
            12          -0.00035        1.000
            13          -0.00023        1.000
            14          -0.00015        1.000
            15          -0.00010        1.000
            16          -0.00007        1.000
            17          -0.00004        1.000
            18          -0.00003        1.000
            19          -0.00002